In [1]:
from scripts.translators import GPT4Client
from scripts.util import MyLogger
gpt1 = GPT4Client(logger=MyLogger(logfile='log.jsonl'))
gpt1.user_prompt = lambda src_lang, tgt_lang, text : text

In [3]:
from scripts.data_management import EPManager
dm = EPManager()
de_sents, en_sents = dm.get_sentence_pairs('de', 'en', num_of_sents=50)


In [4]:
gpt1.translate_document(
    text=de_sents,
    src_lang='de',
    tgt_lang='en'
)

['Resumption of the session period  ',
 'I declare the session period of the European Parliament, which was interrupted on Friday, 17 December, resumed. I would like to once again wish you all the best for the New Year and hope that you had a pleasant holiday.',
 '',
 'As you will have noticed, the much-feared "Millennium Bug" did not occur. However, citizens of some of our Member States have become victims of terrible natural disasters.',
 '',
 'There is a desire in Parliament to hold a debate on this matter during this session period in the coming days.',
 '',
 'Today, I would like to ask you—this is also the wish of some colleagues—to observe a minute of silence in memory of all the victims of the storms, particularly in the various countries of the European Union.',
 '',
 'I ask you to rise for a minute of silence.',
 '(The Parliament rises for a minute of silence.)',
 '',
 'Madam President, on a point of order.',
 'As you will surely know from the press and television, there have 

In [5]:
from scripts.translators import GPT4Client
from scripts.util import MyLogger
gpt2 = GPT4Client(logger=MyLogger(logfile='log.jsonl'))

In [6]:
from scripts.data_management import EPManager
dm = EPManager()
de_sents, en_sents = dm.get_sentence_pairs('de', 'en', num_of_sents=50)

In [7]:
gpt2.translate_document(
    text=de_sents,
    src_lang='de',
    tgt_lang='en'
)

['Resumption of the session period  ',
 'I declare the session period of the European Parliament, which was interrupted on Friday, 17 December, resumed, wish you once again all the best for the New Year, and hope that you had a pleasant holiday.  ',
 'As you have been able to see, the dreaded "Millennium Bug" did not occur. However, citizens of some of our Member States have become victims of terrible natural disasters.  ',
 'There is a desire in Parliament for a debate during this session period in the coming days.  ',
 'Today I would like to ask you – this is also the wish of some colleagues – to commemorate all victims of the storms, especially in the various countries of the European Union, with a minute of silence.  ',
 'I ask you to rise for a minute of silence.  ',
 '(The Parliament rises for a minute of silence.)  ',
 'Madam President, on a point of order.  ',
 'As you surely know from the press and television, there have been several bomb explosions in Sri Lanka with numerous 